In [22]:
#import packages

import re
from geopy.distance import great_circle

In [ ]:
#mathematica names are outputted in camelCase, translating to snake_case.

def camelcase_to_snakecase(name):
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    return re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1).lower()

In [31]:
#infile of weighted population centroids was generated in mathematica
#see weighted-centroids.nb for the code.

wt_centroids_infile = '/Users/alliblk/Desktop/gitrepos/zika-usvi/data/predictors/pop-weighted-centroids.tsv'
wt_centroids_outfile = '/Users/alliblk/Desktop/gitrepos/zika-usvi/data/predictors/great-circle-dists.tsv'

In [15]:
#read in latitudes and longitudes for each country as a dict
#key is the country in snake_case, value is a tuple (latitude,longitude)

weighted_pop_centroids = {}
for line in open(wt_centroids_infile,'rU'):
    place = camelcase_to_snakecase(line.split('\t')[0])
    latitude = float(line.split('\t')[1])
    longitude = float(line.split('\t')[2])
    weighted_pop_centroids[place] = (latitude,longitude)

In [46]:
#do some country editing since mathematica gives us some countries that we aren't interested in.

excluded_countries = ['falkland_islands','montserrat', 'greenland', 'saint_pierre_miquelon']
included_countries = [key for key in weighted_pop_centroids.keys() if key not in excluded_countries]

In [45]:
# write great circle distances to tsv file
# all predictor data is stored as a long edge tsv rather than a matrix, eg
# origin \t destination \t value
# usa         mex           XXX
# mex         usa           ZZZ

with open(wt_centroids_outfile,'w') as file:
    file.write('{}\t{}\t{}\n'.format('origin', 'destination', 'great_circle_dist_km'))
    for origin in included_countries:
        for destination in included_countries:
            dist = great_circle(weighted_pop_centroids[origin],weighted_pop_centroids[destination]).kilometers
            file.write('{}\t{}\t{}\n'.format(origin, destination, dist))